<a href="https://colab.research.google.com/github/rajprakrit/ML-Library-1/blob/master/ANN_ML_Lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
def train_test_split(X, y, ratio):##train test split method

        size = X.shape[0];
        count = size*(1-ratio)
        count = int(count)
        X_train = X[:count]
        y_train = y[:count]
        X_test = X[count:]
        y_test = y[count:]

        return X_train, X_test, y_train, y_test



def sigmoid(X): #sigmoid function

  return 1./(1 + np.exp(-X))

def sigmoid_derv(X): #derivative of sigmoid

  return X * (1 - X)

def softmax(X): #softmax function to generalize output layer such that sum of all outputs is one 

  exponentials = np.exp(X - np.max(X, axis = 1, keepdims = True))

  return exponentials/(np.sum(exponentials, axis = 1, keepdims = True))

def cross_entropy(pred, real): 

    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def error(pred, real): #cross entropy function for error

    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss

class MyANN:

  def __init__(self, X, y, nodes, lr, n_class):
    self.X = X
    self.nodes = nodes
    n_samples, n_features = X.shape
    self.n_class = n_class
    self.lr = lr
    #initialization of parameters of class object
    self.w1 = np.random.randn(n_features, self.nodes)
    self.b1 = np.zeros((1, self.nodes))
    self.w2 = np.random.randn(self.nodes, self.nodes)
    self.b2 = np.zeros((1, self.nodes))
    self.w3 = np.random.randn(self.nodes, self.n_class)
    self.b3 = np.zeros((1, self.n_class))
    self.y = y

  def forwardprop(self):
    #forward propagation using matrix multiplication 
    z1 = np.dot(self.X, self.w1) + self.b1
    self.a1 = sigmoid(z1)
    z2 = np.dot(self.a1, self.w2) + self.b2
    self.a2 = sigmoid(z2)
    z3 = np.dot(self.a2, self.w3) + self.b3
    self.a3 = softmax(z3)

  def backprop(self):
    loss = error(self.a3, self.y)
    print('Error :', loss)
    #calculating delta of every layer using chain rule
    a3_delta = cross_entropy(self.a3, self.y) 
    z2_delta = np.dot(a3_delta, self.w3.T)
    a2_delta = z2_delta * sigmoid_derv(self.a2) 
    z1_delta = np.dot(a2_delta, self.w2.T)
    a1_delta = z1_delta * sigmoid_derv(self.a1) 

    #updating weights between every layer
    self.w3 -= self.lr * np.dot(self.a2.T, a3_delta)
    self.b3 -= self.lr * np.sum(a3_delta, axis=0, keepdims=True)
    self.w2 -= self.lr * np.dot(self.a1.T, a2_delta)
    self.b2 -= self.lr * np.sum(a2_delta, axis=0)
    self.w1 -= self.lr * np.dot(self.X.T, a1_delta)
    self.b1 -= self.lr * np.sum(a1_delta, axis=0)

  def fit(self, n_iters):

    for i in range(n_iters):
      self.forwardprop()
      self.backprop()

  def accuracy(self, test, y):

    self.test = test
    self.forwardprop()
    predicted_label = [ np.argmax(i) for i in self.a3]
    acc = 0
    for i in range(self.test.shape[0]):
      if y[i][predicted_label[i]]==1:
        acc+=1

    acc = acc/self.test.shape[0]

    return acc*100

  def predict(self, test):

    self.test = test
    self.forwardprop()
    

    return [np.argmax(i) for i in self.a3] 

In [0]:
df = pd.read_csv('sample_data/mnist_train_small.csv', header = None)
df1 = pd.read_csv('sample_data/mnist_test.csv', header = None)
X_train = np.array(df.iloc[:, 1:785])
y_train1 = np.array(df.iloc[:, 0])
X_test = np.array(df1.iloc[:, 1:785])
y_test1 = np.array(df1.iloc[:, 0])
def scaling(X):
  return (X - np.mean(X))/(np.max(X) - np.min(X) + 1)

X_test = scaling(X_test)
X_train = scaling(X_train)

In [0]:
y_train = np.zeros([X_train.shape[0], 10])
for i in range(X_train.shape[0]):
  y_train[i][y_train1[i]] = 1

y_test = np.zeros([X_test.shape[0], 10])
for i in range(X_test.shape[0]):
  y_test[i][y_test1[i]] = 1

In [0]:
clf = MyANN(X_train, y_train, 10, 0.5, 10)

In [0]:
clf.fit(1000)

In [0]:
clf.accuracy(X_train, y_train)

85.585

In [0]:
predict = clf.predict(X_test)

In [0]:
[np.argmax(i) for i in predict]

In [0]:
y_test1

array([7, 2, 1, ..., 4, 5, 6])